# How to solve power flow with gigpower
Last modified: 14 June 2021

This notebook will demonstrate how to solve powerflow with gigpower using Newton Raphson and Forward Backward Sweep. It will also show you how to compare solutions with the solution obtained by OpenDSS.

First, install gigpower.

In [ ]:
import sys
!{sys.executable} -m pip install gigpower

## Select a .dss file
Let's use some a simple feeder to demonstrate. You can find some test feeders in `./gigpower/tests/test_feeders`. This folder has modified versions of IEEE feeders.
We'll use the 13 Bus allwye as an example. 

In [ ]:
dss_file = '../gigpower/tests/test_feeders/IEEE_13_Bus_allwye.dss'

## Solve with Newton-Raphson method (NR3)
The following code block creates a SolutionNR3, calls solve, and prints solution values. 

In [ ]:

from gigpower.solution_nr3 import SolutionNR3

my_solution_nr3 = SolutionNR3(dss_file)
my_solution_nr3.solve()
my_solution_nr3.print_solution()

## Solve with Forward Backward Sweep (FBS)
Let's solve the same feeder with Forward Backward Sweep. Note that FBS only works for radial networks. The following code block creates a SolutionFBS, calls solve, and prints solution values. 

In [ ]:
from gigpower.solution_fbs import SolutionFBS

my_solution_fbs = SolutionFBS(dss_file)
my_solution_fbs.solve()
my_solution_fbs.print_solution()

## Solve with OpenDSS
`gigpower.solution_dss` provides a wrapper over OpenDSS.py. Let's use it to solve the same feeder using OpenDSS.

In [ ]:
from gigpower.solution_dss import SolutionDSS

my_solution_dss = SolutionDSS(dss_file)
my_solution_dss.solve()
my_solution_dss.print_solution()

## Get solution dataframes
After calling `.solve()`, can can query Solution objects for the following solution values, as Pandas DataFrames

- Bus Voltage: `Solution.get_V()`
- Bus Voltage Magnitude: `Solution.get_Vmag()`
- Line Current: `Solution.get_I()`
- Line Power, Incoming: `Solution.get_Stx()`
- Line Power, Outgoing: `Solution.get_Srx()`
- Bus Power: `Solution.get_sV()`

Using the `orient` parameter, specify `'rows'` or `'cols'` to get row-major or column-major formats.
The code blocks below get dataframes from our Solution objects, in row-major format.

In [ ]:
my_solution_nr3.get_V(orient='rows')

In [ ]:
my_solution_nr3.get_Vmag(orient='rows')


In [ ]:
my_solution_fbs.get_I(orient='rows')


In [ ]:
my_solution_fbs.get_Stx(orient='rows')


In [ ]:
my_solution_dss.get_Stx(orient='rows')

In [ ]:
my_solution_dss.get_sV(orient='')

## Compare Solutions
`gigpower.pretty_print` provides a method for printing a useful comparison between two Solution objects. The code block below compares `my_solution_nr3` with `my_solution_fbs`. 

In [ ]:
from gigpower.pretty_print import compare_solutions
compare_solutions(my_solution_nr3, my_solution_fbs, 'NR3', 'FBS')


## Zip Values

Zip Values are set **class-wide** on all Solutions. The default Zip Values are [0.1, 0.05, 0.85, 0.1, 0.05, 0.85,0.8]. Because the Zip Values are class variables, if you change them for one Solution you change them for all Solutions! So if you do change zip values, be sure to call `solve()` again on each Solution in scope. 

The code blocks below demonstrate getting and setting zip values.  

In [ ]:
my_solution_dss.ZIP_V

In [ ]:
# after running this cell, all Solution zip values will be re-set.
my_solution_nr3.set_zip_values([1,0,0,1,0,0,.8])

In [ ]:
my_solution_dss.ZIP_V

## Other Solution API methods
The code snippets below demonstrate other useful Solution getters and setters.

In [ ]:
# get nominal bus powers (you don't need to call solve() first for this)
my_solution_nr3.get_nominal_bus_powers()

In [ ]:
# get Solution instance parameters
my_solution_fbs.get_params()

In [ ]:
# View Solution class parameters - these are all the variables that the Solution will solve for, their axes, and their datatypes. 
my_solution_nr3.SOLUTION_PARAMS

In [ ]:
# The SolutionFBS objects provide an adjacency matrix, with the convention that edges are directed away from the source bus. 
my_solution_fbs.adj

In [ ]:
# You can get the reverse adjacency matrix to view edges directed towards source.
my_solution_fbs.reverse_adj